# Mental Health in Tech
Используя данные опросов Open Source Mental Illness (OSMI) за период с 2014 по 2019 год, исследование будет направлено на изучение особенностей психического здоровья в сфере технологий. Анализируя отношение к психическому здоровью и частоту нарушений психического состояния на рабочем месте, можно выявить тенденции и изменения в течение этих лет.
- Ссылка на датасет: [Mental Health in Tech](https://www.kaggle.com/datasets/anth7310/mental-health-in-the-tech-industry)

In [4]:
%pip install plotly

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/a8/07/72953cf70e3bd3a24cbc3e743e6f8539abe6e3e6d83c3c0c83426eaffd39/plotly-5.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e454214013b441c4575/tenacity-8.2.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.6 MB 1.2 MB/s eta 0:00:13
    --------------------------------------- 0.3/15.6 MB 2.3 MB/s eta 0:00:07
   - -------------------------------------- 0.5/15.6 MB 3.1 MB/s eta 0:00:05
   -- ------------------------------------- 0.9/15.6 MB 3.9 MB/s eta 0:00:04
   -- ------------------------------------- 0.9/15.6 MB 3.9 MB/s eta 0:00:04
   --- ------------------------------------ 1.6/15.6 MB 5.2 MB/s eta 


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import sqlite3
import pandas as pd
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots

In [135]:
con = sqlite3.connect('first', timeout=10)
cur = con.cursor()

con1 = sqlite3.connect('data\mental_health.sqlite', timeout=10)
cur1 = con.cursor()

In [136]:
answer_df = pd.read_sql_query("SELECT * from answer", con1)
question_df = pd.read_sql_query("SELECT * from question", con1)
survey_df = pd.read_sql_query("SELECT * from survey", con1)

In [588]:
copy_question_df = question_df.copy()
copy_question_df.columns = ['question_text', 'question_id']

copy_survey_df = survey_df.copy()
copy_survey_df.columns = ['survey_id', 'description']

copy_answer_df = answer_df.copy()
copy_answer_df.columns = ['answer_text','survey_id', 'user_id', 'question_id']

In [590]:
def gender(x):
    if ('female' in x.lower() or 'she' in x.lower()):
        return 'female'
    elif 'male' in x.lower():
        return 'male'
    elif x.lower() == '-1':
        return '-1'
    else: return 'other'

def contry_USA(x):
    if 'United States of America' in x:
        return 'United States' 
    return x

In [591]:
new_gender = copy_answer_df[copy_answer_df['question_id'] == 2]['answer_text'].map(gender)
copy_answer_df.loc[copy_answer_df.question_id == 2, 'answer_text'] = new_gender

In [592]:
new_contry = copy_answer_df[copy_answer_df['question_id'] == 3]['answer_text'].map(contry_USA)
copy_answer_df.loc[copy_answer_df.question_id == 3, 'answer_text'] = new_contry

In [595]:

copy_question_df.to_sql(con=con, name='question', index=False, if_exists='replace')
copy_survey_df.to_sql(con=con, name='survey', index=False, if_exists='replace')
copy_answer_df.to_sql(con=con, name='answer', index=False, if_exists='replace')

236898

In [ ]:
# Количество вопросов по годам
'''
SELECT  a.survey_id,
		COUNT(DISTINCT(a.question_id)) AS question_amount
FROM answer a 
GROUP BY 1
'''
# Сколько людей прошло опрос в каждом году
'''
SELECT  a.survey_id,
		COUNT(DISTINCT(a.user_id)) AS participants_amount
FROM answer a 
GROUP BY 1
'''

# Количество людей из разных возрстых групп по годам
'''
SELECT a.survey_id, 
       CASE 
            WHEN a.answer_text BETWEEN 18 AND 24 THEN '18-24'
            WHEN a.answer_text BETWEEN 25 AND 34 THEN '25-34'
            WHEN a.answer_text BETWEEN 35 AND 44 THEN '35-44'
            WHEN a.answer_text BETWEEN 45 AND 54 THEN '45-54'
            WHEN a.answer_text BETWEEN 55 AND 64 THEN '55-64'
            WHEN a.answer_text BETWEEN 65 AND 99 THEN 'Over 65 '
        END AS age_group,  
    COUNT(a.user_id) AS count
FROM answer a 
WHERE a.question_id = 1 AND a.answer_text > 18   
GROUP BY a.survey_id, age_group
ORDER BY a.survey_id, age_group;  
'''

# Сколько мужчин и женщин принимали участие в опросе в каждом году
'''
SELECT  a.survey_id, 
		a.answer_text,
		COUNT(DISTINCT(a.user_id)) AS amount
FROM answer a 
WHERE a.question_id = 2 AND (a.answer_text = 'female' OR a.answer_text = 'male')
GROUP BY a.survey_id, a.answer_text
ORDER BY a.survey_id, a.answer_text; 
'''

# Из каких стран 
'''
SELECT  a.answer_text,
		COUNT(DISTINCT(a.user_id)) AS amount
FROM answer a 
WHERE a.question_id = 3
GROUP BY a.answer_text
ORDER BY 2 DESC;  
'''

# Сколько людей имеют психические заболевания в год опроса
'''
SELECT 	a.survey_id,
		a.answer_text,
		COUNT(DISTINCT(a.user_id)) AS have_disorder
FROM answer a 
WHERE a.question_id = 33 AND a.answer_text = 'Yes'
GROUP BY 1
ORDER BY 1
'''

# Сколько процентов ответило что имеют псих заболевание 
'''
WITH aa AS (
SELECT 	a.survey_id,
		a.answer_text,
		a.user_id
FROM answer a
WHERE a.question_id = 33
ORDER BY 1, 2
)

SELECT  aa.survey_id,
		COUNT(DISTINCT(aa.user_id)) AS amount_part,
		COUNT(IIF(aa.answer_text = 'Yes', 1, NULL)) AS have_disorder,
		(100 * COUNT(IIF(aa.answer_text = 'Yes', 1, NULL))) / COUNT(DISTINCT(aa.user_id)) AS percent_have_disorder
FROM aa
GROUP BY 1
'''

# Какие заболевания были тех кто ответил что имеют псих заболевание, какой процент от общего числа участников и от тех кто ответил что имеют псих заболевание 
'''
WITH aa115 AS (
SELECT 	a.survey_id,
		a.answer_text,
		a.user_id,
		*
FROM answer a
WHERE a.question_id = 115
ORDER BY 3 DESC
),

aa33 AS (
SELECT 	a.survey_id,
		a.answer_text,
		a.user_id
FROM answer a
WHERE a.question_id = 33 AND a.answer_text = 'Yes' AND a.survey_id = 2016
ORDER BY 3 DESC
)

SELECT  aa115.answer_text,
		COUNT(DISTINCT(aa115.user_id)) AS amount_disorder,
		(100* COUNT(DISTINCT(aa115.user_id))) / total_amount AS percent_from_total,
		(100* COUNT(DISTINCT(aa115.user_id))) / total_amount_have_disorder AS percent_from_have_disorder
FROM 	aa115,
		(SELECT count(DISTINCT(aa115.user_id)) AS total_amount FROM aa115),
		(SELECT count(DISTINCT(aa33.user_id)) AS total_amount_have_disorder FROM aa33)
LEFT JOIN aa33
ON aa115.user_id = aa33.user_id
WHERE aa33.answer_text = 'Yes'
GROUP BY 1
ORDER BY 2 DESC 
'''

# Кем работают те кто ответил что имеют псих заболевание, какой процент от тех кто ответил что имеют псих заболевание 

'''
WITH aa117 AS (
SELECT 	a.survey_id,
		a.answer_text,
		a.user_id,
		*
FROM answer a
WHERE a.question_id = 117
ORDER BY 3 DESC
),

aa33 AS (
SELECT 	a.survey_id,
		a.answer_text,
		a.user_id
FROM answer a
WHERE a.question_id = 33 AND a.answer_text = 'Yes' AND a.survey_id = 2016
ORDER BY 3 DESC
)

SELECT  aa117.answer_text,
		COUNT(DISTINCT(aa33.user_id)) AS amount_have_disorder,
		(100* COUNT(DISTINCT(aa33.user_id))) / total_amount_have_disorder AS percent_from_have_disorder
FROM 	aa117,
		(SELECT count(DISTINCT(aa33.user_id)) AS total_amount_have_disorder FROM aa33)
LEFT JOIN aa33
ON aa117.user_id = aa33.user_id
GROUP BY 1
ORDER BY 2 DESC 
'''

